In [15]:
#import!
from IPython.utils import io
import pybullet as p
import pybullet_data
from time import sleep
import pyrosim.pyrosim as pyrosim
import numpy as np
import random
import math

In [2]:
import os
def npsave(name, data):
    i=1
    while(os.path.exists(f'{name}{i}.npy')): i+=1
    np.save(f'{name}{i}', data)
    print(f'Numpy ndarray saved: {name}{i}.npy')

In [48]:
#with io.capture_output() as captured:
physicsClient = p.connect(p.GUI)
p.setAdditionalSearchPath(pybullet_data.getDataPath())

startThreads creating 1 threads.
starting thread 0
started thread 0 
argc=2
argv[0] = --unused
argv[1] = --start_demo_name=Physics Server
ExampleBrowserThreadFunc started
X11 functions dynamically loaded using dlopen/dlsym OK!
X11 functions dynamically loaded using dlopen/dlsym OK!
Creating context
Created GL 3.3 context
Direct GLX rendering context obtained
Making context current
GL_VENDOR=Microsoft Corporation
GL_RENDERER=D3D12 (NVIDIA GeForce RTX 2060)
GL_VERSION=3.3 (Core Profile) Mesa 21.0.3
GL_SHADING_LANGUAGE_VERSION=3.30
pthread_getconcurrency()=0
Version = 3.3 (Core Profile) Mesa 21.0.3
Vendor = Microsoft Corporation
Renderer = D3D12 (NVIDIA GeForce RTX 2060)
b3Printf: Selected demo: Physics Server
startThreads creating 1 threads.
starting thread 0
started thread 0 
MotionThreadFunc thread started


In [49]:
iterations = 1000

p.setGravity(0,0,-9.80665)
p.loadSDF("world.sdf")
plane = p.loadURDF("plane.urdf")
robot = p.loadURDF("body.urdf")
backLegSensorValues = np.zeros(iterations)
frontLegSensorValues = np.zeros(iterations)

b3Printf: b3Warning[examples/Importers/ImportURDFDemo/BulletUrdfImporter.cpp,126]:

b3Printf: expected a world element, continuing without it.


In [45]:
amplitude, frequency, phaseOffset = math.pi/4, 20, 0
targetAngles = np.sin(np.linspace(0, 2*math.pi, num=iterations)*frequency + phaseOffset)*amplitude

In [51]:
pyrosim.Prepare_To_Simulate(robot)
for i in range(iterations):
    p.stepSimulation()
    backLegSensorValues[i] = pyrosim.Get_Touch_Sensor_Value_For_Link("BackLeg")
    frontLegSensorValues[i] = pyrosim.Get_Touch_Sensor_Value_For_Link("FrontLeg")
    # print(backLegTouch)
    pyrosim.Set_Motor_For_Joint(bodyIndex = robot,
                            jointName = b"Torso_BackLeg",
                            controlMode = p.POSITION_CONTROL,
                            targetPosition = targetAngles[i],
                            maxForce = 50)
    pyrosim.Set_Motor_For_Joint(bodyIndex = robot,
                            jointName = b"Torso_FrontLeg",
                            controlMode = p.POSITION_CONTROL,
                            targetPosition =  0.5*targetAngles[int((i+iterations/80)%iterations)],
                            maxForce = 50)
    sleep(1/60)

In [52]:
p.disconnect()

numActiveThreads = 0
stopping threads
Thread with taskId 0 exiting
Thread TERMINATED
destroy semaphore
semaphore destroyed
destroy main semaphore
main semaphore destroyed
finished
numActiveThreads = 0
btShutDownExampleBrowser stopping threads
Thread with taskId 0 exiting
Thread TERMINATED
destroy semaphore
semaphore destroyed
destroy main semaphore
main semaphore destroyed


In [ ]:
npsave('LegSensorValues', np.column_stack((backLegSensorValues, frontLegSensorValues)))

Numpy ndarray saved: LegSensorValues1.npy
